In [5]:
import numpy as np

def I(valor, total):
    valor = int(valor)
    total = int(total)
    if valor <= 0 or total <= 0 or valor > total:
        return 0
    else:
        return -valor / total * np.log2(valor / total)


In [6]:
import pandas as pd
import math
from collections import Counter
import treelib

def C45(df, coluna_classe):
    total = len(df)
    entropia_classe = 0

    # Entropia da classe (target)
    for classe, freq in df[coluna_classe].value_counts().items():
        entropia_classe += I(freq, total)

    melhor_atributo = [0, None]

    for coluna in df.columns:
        if coluna == coluna_classe:
            continue

        entropia_atributo = 0
        split_info = 0
        valores = df[coluna].value_counts()

        for valor, qtd in valores.items():
            subconjunto = df[df[coluna] == valor]
            proporcao = qtd / total

            entropia_sub = 0
            for classe, freq in subconjunto[coluna_classe].value_counts().items():
                entropia_sub += I(freq, qtd)

            entropia_atributo += proporcao * entropia_sub
            split_info += I(qtd, total)  # Split Info é a entropia dos valores do atributo

        ganho_info = entropia_classe - entropia_atributo

        if split_info != 0:
            gain_ratio = ganho_info / split_info
        else:
            gain_ratio = 0

        if gain_ratio > melhor_atributo[0]:
            melhor_atributo = [gain_ratio, coluna]

    return melhor_atributo


In [7]:
import pandas as pd
import math
from collections import Counter
import treelib

def geraArvore(df, coluna_classe, tree, parent=None, ramo_valor=None, nivel_atual=0, altura_maxima=5):
    # Limite de altura atingido → criar nó folha com classe majoritária
    if nivel_atual >= altura_maxima:
        classe_majoritaria = str(df[coluna_classe].mode()[0])
        folha_id = f"{classe_majoritaria}_{len(tree.nodes)}"
        tag = f"{ramo_valor} → {classe_majoritaria}" if ramo_valor else classe_majoritaria
        tree.create_node(tag=tag, identifier=folha_id, parent=str(parent))
        return

    melhorAtributo = C45(df, coluna_classe)

    if melhorAtributo[0] != 0:
        atributo = str(melhorAtributo[1])
        if parent is None:
            node_id = str(atributo)
            tree.create_node(tag=str(atributo), identifier=node_id)
        else:
            node_id = f"{atributo}_{len(tree.nodes)}"
            tree.create_node(tag=f"{ramo_valor} → {atributo}", identifier=node_id, parent=str(parent))

        for valor in df[atributo].unique():
            df_filtrado = df[df[atributo] == valor]
            classes = df_filtrado[coluna_classe].unique()
            if len(classes) == 1:
                classe_final = str(classes[0])
                folha_id = f"{classe_final}_{len(tree.nodes)}"
                tree.create_node(tag=f"{valor} → {classe_final}", identifier=folha_id, parent=node_id)
            else:
                df_sub = df_filtrado.drop(columns=[atributo])
                # chamada recursiva com incremento de profundidade
                geraArvore(df_sub, coluna_classe, tree, parent=node_id, ramo_valor=str(valor), nivel_atual=nivel_atual + 1, altura_maxima=altura_maxima)
    else:
        classe_majoritaria = str(df[coluna_classe].mode()[0])
        folha_id = f"{classe_majoritaria}_{len(tree.nodes)}"
        tag = f"{ramo_valor} → {classe_majoritaria}" if ramo_valor else classe_majoritaria
        tree.create_node(tag=tag, identifier=folha_id, parent=str(parent))


In [8]:
import pandas as pd
import math
from collections import Counter
from treelib import Tree


# Abrindo o arquivo CSV
df = pd.read_csv('train.csv',sep = ",")

# Remover colunas que não são úteis para classificação
df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], inplace=True)

# Preencher valores ausentes:
# 'Age' será preenchido com a mediana
df['Age'].fillna(df['Age'].median(), inplace=True)

# 'Embarked' será preenchido com o valor mais frequente
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Convertendo variáveis categóricas em numéricas ou discretas:

# 'Sex' (categórica para nominal)
df['Sex'] = df['Sex'].map({'male': 'Homem', 'female': 'Mulher'})

# Discretizar a idade em faixas
df['Age'] = pd.cut(df['Age'], bins=[0, 12, 18, 30, 50, 80], labels=['Criança', 'Adolescente', 'Jovem', 'Adulto', 'Idoso'])

# Discretizar a tarifa (Fare)
df['Fare'] = pd.qcut(df['Fare'], q=4, labels=['Baixo', 'Médio', 'Alto', 'Muito alto'])

# Mapear 'Embarked' para nomes
df['Embarked'] = df['Embarked'].map({'S': 'Southampton', 'C': 'Cherbourg', 'Q': 'Queenstown'})

coluna_classe = "Survived"
# TEste de abertura
#print(df.head()) 
  
#colocar na árvore
tree = Tree()
geraArvore(df, coluna_classe, tree, parent=None, nivel_atual=0, altura_maxima=3)
tree.show()

Sex
├── Homem → Pclass
│   ├── 1 → Age
│   │   ├── Adolescente → 0
│   │   ├── Adulto → 0
│   │   ├── Criança → 1
│   │   ├── Idoso → 0
│   │   └── Jovem → 0
│   ├── 2 → Age
│   │   ├── Adolescente → 0
│   │   ├── Adulto → 0
│   │   ├── Criança → 1
│   │   ├── Idoso → 0
│   │   └── Jovem → 0
│   └── 3 → Age
│       ├── Adolescente → 0
│       ├── Adulto → 0
│       ├── Criança → 0
│       ├── Idoso → 0
│       └── Jovem → 0
└── Mulher → Pclass
    ├── 1 → Age
    │   ├── Adolescente → 1
    │   ├── Adulto → 1
    │   ├── Criança → 0
    │   ├── Idoso → 1
    │   └── Jovem → 1
    ├── 2 → Embarked
    │   ├── Cherbourg → 1
    │   ├── Queenstown → 1
    │   └── Southampton → 1
    └── 3 → Embarked
        ├── Cherbourg → 1
        ├── Queenstown → 1
        └── Southampton → 0



C:\Users\Eu\AppData\Local\Temp\ipykernel_17804\585159572.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
C:\Users\Eu\AppData\Local\Temp\ipykernel_17804\585159572.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example